# Kiva Loan Type Prediction

In [125]:
#importing library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as  sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as  tf
from sklearn.metrics import confusion_matrix,classification_report

In [126]:
#loading the dataset
df=pd.read_csv('../input/data-science-for-good-kiva-crowdfunding/kiva_loans.csv')
#loadig the dataset
df

,id,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,partner_id,posted_time,disbursed_time,funded_time,term_in_months,lender_count,tags,borrower_genders,repayment_interval,date
0,653051,300.0,300.0,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:12:39+00:00,2013-12-17 08:00:00+00:00,2014-01-02 10:06:32+00:00,12.0,12,NaN,female,irregular,2014-01-01
1,653053,575.0,575.0,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:51:08+00:00,2013-12-17 08:00:00+00:00,2014-01-02 09:17:23+00:00,11.0,14,NaN,"female, female",irregular,2014-01-01
2,653068,150.0,150.0,Transportation,Transportation,To repair their old cycle-van and buy another ...,IN,India,Maynaguri,INR,334.0,2014-01-01 09:58:07+00:00,2013-12-17 08:00:00+00:00,2014-01-01 16:01:36+00:00,43.0,6,"user_favorite, user_favorite",female,bullet,2014-01-01
3,653063,200.0,200.0,Embroidery,Arts,to purchase an embroidery machine and a variet...,PK,Pakistan,Lahore,PKR,247.0,2014-01-01 08:03:11+00:00,2013-12-24 08:00:00+00:00,2014-01-01 13:00:00+00:00,11.0,8,NaN,female,irregular,2014-01-01
4,653084,400.0,400.0,Milk Sales,Food,to purchase one buffalo.,PK,Pakistan,Abdul Hakeem,PKR,245.0,2014-01-01 11:53:19+00:00,2013-12-17 08:00:00+00:00,2014-01-01 19:18:51+00:00,14.0,16,NaN,female,monthly,2014-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671200,1340323,0.0,25.0,Livestock,Agriculture,"[True, u'para compara: cemento, arenya y ladri...",PY,Paraguay,Concepción,USD,58.0,2017-07-25 16:55:34+00:00,2017-07-25 07:00:00+00:00,NaN,13.0,0,NaN,female,monthly,2017-07-25
671201,1340316,25.0,25.0,Livestock,Agriculture,"[True, u'to start a turducken farm.'] - this l...",KE,Kenya,NaN,KES,138.0,2017-07-25 06:14:08+00:00,2017-07-24 07:00:00+00:00,2017-07-26 02:09:43+00:00,13.0,1,NaN,female,monthly,2017-07-25
671202,1340334,0.0,25.0,Games,Entertainment,NaN,KE,Kenya,NaN,KES,138.0,2017-07-26 00:02:07+00:00,2017-07-25 07:00:00+00:00,NaN,13.0,0,NaN,NaN,monthly,2017-07-26
671203,1340338,0.0,25.0,Livestock,Agriculture,"[True, u'to start a turducken farm.'] - this l...",KE,Kenya,NaN,KES,138.0,2017-07-26 06:12:55+00:00,2017-07-25 07:00:00+00:00,NaN,13.0,0,NaN,female,monthly,2017-07-26


In [127]:
#getting information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671205 entries, 0 to 671204
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  671205 non-null  int64  
 1   funded_amount       671205 non-null  float64
 2   loan_amount         671205 non-null  float64
 3   activity            671205 non-null  object 
 4   sector              671205 non-null  object 
 5   use                 666973 non-null  object 
 6   country_code        671197 non-null  object 
 7   country             671205 non-null  object 
 8   region              614405 non-null  object 
 9   currency            671205 non-null  object 
 10  partner_id          657698 non-null  float64
 11  posted_time         671205 non-null  object 
 12  disbursed_time      668809 non-null  object 
 13  funded_time         622874 non-null  object 
 14  term_in_months      671205 non-null  float64
 15  lender_count        671205 non-nul

In [128]:
#encoding date
def encode_dates(df,column):
    df=df.copy()
    #converting date column to datetime object
    df[column]=pd.to_datetime(df[column])
    #creating year column from the date column
    df[column+'_year']=df[column].apply(lambda x:x.year)
    #creating month column from the  date column
    df[column+'_month']=df[column].apply(lambda x:x.month)
    #creating day column frm the date column
    df[column+'_day']=df[column].apply(lambda x:x.day)
    #dropping the original date column
    df=df.drop(column,axis=1)
    #returning the dataframe
    return df
    

In [129]:
#calculating the number of female in gender column

z='male, female, female, female, female, female, female, female, male, male, female, male, female, male, male, male'
z.split(',')

['male',
 ' female',
 ' female',
 ' female',
 ' female',
 ' female',
 ' female',
 ' female',
 ' male',
 ' male',
 ' female',
 ' male',
 ' female',
 ' male',
 ' male',
 ' male']

In [130]:
#creating one hot encode function
def onehot_encode(df,columns_with_prefixes):
    df=df.copy()
    for column,prefix in columns_with_prefixes:
        dummies=pd.get_dummies(df[column],prefix=prefix)
        df=pd.concat([dummies,df],axis=1)
        df=df.drop(column,axis=1)
    return df
    

In [131]:
#
def get_male_count(x):
    count=0
    for gender in str(x).split(','):
        if gender=='male':
            count+=1
    return count
def get_female_count(x):
    count=0
    for gender in str(x).split(','):
        if gender=='female':
            count+=1
    return count
    

In [132]:
#creating preprocessing function
def preprocess_inputs(df):
    df=df.copy()
    #dropping id column
    df=df.drop('id',axis=1)
    #dropping tag and use column
    df=df.drop(['tags','use'],axis=1)
    #removing country and date column
    df=df.drop(['country','date'],axis=1)
    #Droping region column high-cardinality
    df=df.drop('region',axis=1)
    #implementing encode_dates function
    df=encode_dates(df,'posted_time')
    df=encode_dates(df,'disbursed_time')
    df=encode_dates(df,'funded_time')
    #applying male count function to sum to number of male of each row
    df['male_count']=df['borrower_genders'].apply(get_male_count)
    #applying female count  function to sum to number of female on each row
    df['female_count']=df['borrower_genders'].apply(get_female_count)
    #dropping borrower_genders
    df=df.drop('borrower_genders',axis=1)
    nominal_features=[
        ('activity','act'),
        ('sector','sec'),
        ('country_code','ctc'),
        ('currency','curr')
    ]
    df=onehot_encode(df,nominal_features)
    #now spliting the dataset
    Y=df['repayment_interval']
    X=df.drop('repayment_interval',axis=1)
    #Encode labels
    label_mapping={
        'bullet':0,
        'weekly':1,
        'monthly':2,
        'irregular':3
    }
    #replacing target column with label_mapping
    Y=Y.replace(label_mapping)
    #filling in remaining missing vlaues with column  mean
    missing_value_columns=X.loc[:,X.isna().sum()>0].columns
    for column in missing_value_columns:
        X[column]=X[column].fillna(X[column].mean())
    #train_test_split
    x_train,x_test,y_train,y_test=train_test_split(X,Y,train_size=0.7,random_state=1)
    #Scaling X with a standard Scaler
    scaler=StandardScaler()
    scaler.fit(x_train)
    x_train=pd.DataFrame(scaler.transform(x_train),columns=X.columns)
    x_test= pd.DataFrame(scaler.transform(x_test),columns=X.columns)
   
    return x_train,x_test,y_train,y_test

In [133]:
x_train

,curr_ALL,curr_AMD,curr_AZN,curr_BIF,curr_BOB,curr_BRL,curr_BZD,curr_CLP,curr_CNY,curr_COP,...,posted_time_month,posted_time_day,disbursed_time_year,disbursed_time_month,disbursed_time_day,funded_time_year,funded_time_month,funded_time_day,male_count,female_count
0,-0.054629,-0.09503,-0.054216,-0.035967,-0.105222,-0.019413,-0.01353,-0.002527,-0.013918,-0.18356,...,-0.641361,-1.108166,0.612006,-0.900609,0.106113,0.591513,-0.690173,0.239630,-0.544036,0.553702
1,-0.054629,-0.09503,-0.054216,-0.035967,-0.105222,-0.019413,-0.01353,-0.002527,-0.013918,-0.18356,...,0.544460,-1.691507,0.612006,0.270950,-0.606847,0.591513,0.529131,-0.634446,-0.544036,0.553702
2,-0.054629,-0.09503,-0.054216,-0.035967,-0.105222,-0.019413,-0.01353,-0.002527,-0.013918,-0.18356,...,-0.641361,0.991863,-0.332442,-0.607719,0.581420,-0.394198,-0.385347,-1.883126,-0.544036,0.553702
3,-0.054629,-0.09503,-0.054216,-0.035967,-0.105222,-0.019413,-0.01353,-0.002527,-0.013918,-0.18356,...,1.730281,-0.874829,-1.276891,1.442510,0.937900,-1.379908,1.748434,-0.384710,-0.544036,0.553702
4,-0.054629,-0.09503,-0.054216,-0.035967,-0.105222,-0.019413,-0.01353,-0.002527,-0.013918,-0.18356,...,-1.234272,0.758526,-0.332442,-1.193499,-0.131540,-0.394198,-0.994999,-1.758258,1.838112,-1.806026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469838,-0.054629,-0.09503,-0.054216,-0.035967,-0.105222,-0.019413,-0.01353,-0.002527,-0.013918,-0.18356,...,-1.234272,-1.458171,0.612006,-1.486388,0.106113,0.591513,-1.299825,-1.383654,-0.544036,0.553702
469839,-0.054629,-0.09503,-0.054216,-0.035967,-0.105222,-0.019413,-0.01353,-0.002527,-0.013918,-0.18356,...,0.840915,-0.058151,0.612006,0.856730,-0.963328,0.591513,0.833957,1.113706,-0.544036,0.553702
469840,-0.054629,-0.09503,-0.054216,-0.035967,-0.105222,-0.019413,-0.01353,-0.002527,-0.013918,-0.18356,...,0.544460,-0.874829,0.612006,0.270950,1.056727,0.591513,0.833957,-0.384710,-0.544036,0.553702
469841,-0.054629,-0.09503,-0.054216,-0.035967,-0.105222,-0.019413,-0.01353,-0.002527,-0.013918,-0.18356,...,0.840915,-0.058151,0.612006,0.563840,0.937900,0.591513,1.138783,0.614234,-0.544036,0.553702


In [134]:
(X.isna().sum()>0).value_counts()

False    347
dtype: int64

In [135]:
df['repayment_interval'].unique()

array(['irregular', 'bullet', 'monthly', 'weekly'], dtype=object)

In [136]:
#calling the function
x_train,x_test,y_train,y_test=preprocess_inputs(df)
print(x_train.shape)
print(x_test.shape)

(469843, 347)
(201362, 347)


In [ ]:
#checking for missing value in each column
df.isna().mean()

In [138]:
#getting unique value in each column for categorical columns
{column:len(x[column].unique()) for column in x.select_dtypes('object').columns}

AttributeError: 'KerasTensor' object has no attribute 'select_dtypes'

In [ ]:
x['borrower_genders'].unique()

In [ ]:
pd.get_dummies()

In [ ]:
#converting to date time column
pd.to_datetime(x['posted_time'])

In [ ]:
#Trainig the model
inputs=tf.keras.Input(shape=(x_train.shape[1],))
x=tf.keras.layers.Dense(128,activation='relu')(inputs)
x=tf.keras.layers.Dense(128,activation='relu')(x)
outputs=tf.keras.layers.Dense(4,activation='softmax')(x)
model=tf.keras.Model(inputs=inputs,outputs=outputs)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[
            tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=3,
                                            restore_best_weights=True)
    ])

In [ ]:
def evaluate_model(model,x_test,y_test):
    result=model.evaluate(x_test,y_test,verbose=0)
    print('Test Score {:4f}'.format(result[0]))
    print('Test Acuracy:{:2f}'.format(result[1]*100))
    y_pred=np.max(model.predict(x_test),axis=1)
    cm=confusion_matix(y_test,y_pred)
    clr=classification_report(y_test,y_pred,target_names=['bullet','weekly','monthly','irregular'])
    plt.figure(figsize=(10,10))
    sns.heatmap(cm,annot=True,fmt='g',cmap='Blues',cbar=False)
    plt.xticks(np.arange(4)*0.5)